# Standard Bank Tech Impact Challenge: Xente credit scoring challenge 

In this challenge, the task is to predict from a historical dataset of transactions and repayments the likelihood of default in each transaction in a test set contaning various customers.
This is a binary classification problem where the output should be 1 for a defaulted loan and 0 otherwise.

In [1]:
## importing libraries
import pandas as pd
import numpy as np

In [2]:
## reading the files and loading them into dataframes.
train = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Train.csv')
test= pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Test.csv')
sample = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/sample_submission.csv')
mask = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/unlinked_masked_final.csv')
variabs = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/VariableDefinitions.csv')

I'm still not quite familiar with the nature of the data, and how data needs to be organized in order to properly model for this challenge. I will update this notebook as soon as i get better with this particular problem.

In [3]:
## The data was ordered by date. So lets check it.
train.head(3)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
0,CustomerId_27,2018-09-21 12:17:39,550.0,-550.0,TransactionId_1683,BatchId_641,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CustomerId_27,2018-09-25 09:20:29,550.0,-550.0,TransactionId_2235,BatchId_820,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CustomerId_27,2018-09-25 10:33:31,550.0,-550.0,TransactionId_1053,BatchId_210,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So it starts from 21th of September 2018 **( Do you remember 21st night of September? Customer_27 does. Couldn't help it sorry)**

In [4]:
## It ends 31th of March 2019
train.tail(3)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
2097,CustomerId_34,2019-03-31 05:40:10,5000.0,-5000.0,TransactionId_2507,BatchId_1093,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1585,2019-06-15 06:20:05,1.0,InvestorId_1,2019-04-30 05:40:02,LoanApplicationId_1466,PayBackId_1271,ThirdPartyId_1981,1.0,0.0
2098,CustomerId_346,2019-03-31 09:37:19,1500.0,-1500.0,TransactionId_540,BatchId_1239,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1314,2019-04-23 15:44:15,1.0,InvestorId_1,2019-04-30 09:37:15,LoanApplicationId_1389,PayBackId_945,ThirdPartyId_705,1.0,0.0
2099,CustomerId_188,2019-03-31 12:16:33,1000.0,-1000.0,TransactionId_1889,BatchId_355,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1570,2019-04-10 17:33:19,1.0,InvestorId_1,2019-04-30 12:16:26,LoanApplicationId_57,PayBackId_830,ThirdPartyId_899,1.0,0.0


It says on the challenge data section that the maximum date is 17 July 2019. So we can logically assume that we're being tested on transactions between 01-04-2019 and 17-07-2019

In [5]:
## Lets check that
print(test.TransactionStartTime.min(),test.TransactionStartTime.max())

2019-03-31 13:33:05 2019-07-17 05:34:45


Actually the testing starts the same day train dates end. The 31th of March.

###### Definition of loan default

Loan default occurs when a borrower fails to pay back a debt according to the initial arrangement. 
In the case of most consumer loans, this means that successive payments have been missed over the course of weeks or months. 
Fortunately, lenders and loan servicers usually allow a grace period before penalizing the borrower after missing one payment. 
The period between missing a loan payment and having the loan default is known as delinquency. 
The delinquency period gives the debtor time to avoid default by contacting their loan servicer or making up missed payments.

So there's a grace period in which the date on which the borrower is set to pay the loan or rather a portion of it is exceeded. The loan doesn't default immediately, that period is called deliquency. It should be a feature to extract and would be a good one logically. If someone is not defaulting on loans, but is missing a lot of payments he should be more likely to default soon.

**In the context of this competition and the nature of the loans granted, the grace period is 30 days!**

In [40]:
## lets see if the grace period of 30 days is true for this data
train[(train.PaidOnDate>train.DueDate)].IsDefaulted.value_counts()

0.0    341
1.0     75
Name: IsDefaulted, dtype: int64

**Something is fishy.. Not all the transactions marked as Defaulted are in this dataframe. Is it possible to have transactions that were paid before the duedate and are still marked as Defaulted? lets explore that**

In [46]:
len(train[(train.PaidOnDate<train.DueDate)&(train.IsDefaulted==1)])

103

In [44]:
## Taking the example of the LoanId_1376
train[train.LoanId=='LoanId_1376']

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
1847,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-04-02 10:04:20,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_449,ThirdPartyId_216,1.0,1.0
1848,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-04-02 20:25:15,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_843,ThirdPartyId_169,1.0,1.0
1849,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-05-20 08:15:05,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_1533,ThirdPartyId_1915,1.0,1.0
1850,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-05-28 11:05:04,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_443,ThirdPartyId_460,1.0,1.0


***Alright, i can make some sense out of it. Since a single loan is paid by multiple transactions, when a loan defaults, all the transactions associated with that loan are marked as Defaulted. In the example above, you can see some payments ( PaidOnDate ) were made sooner than the DueDate but 2 associated payments were made much later which made the whole loan default. And all transactions associated to that loan are defaulted.***